<a href="https://colab.research.google.com/github/yucongo/ChatterBot/blob/master/Neural_Machine_Translation_with_Seq2Seq_in_Keras_Malik_22nd_article_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 22n Neural Machine Translation with Seq2Seq in Keras
Usman Malik • October 09, 2019
https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/

In [0]:
'''
refer also to win10 playground\neural-machine-translation
'''
import os, sys

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Path("/content/drive/My Drive/data").exists()
# !ls '/content/drive/My Drive/data'
# !mkdir -p /contend/drive

In [0]:
# https://gist.github.com/joshbode/58fac7ababc700f51e2a9ecdebe563ad
import sys
import logging
from typing import Optional, Dict

try:
  import colorama
except ModuleNotFoundError:
  get_ipython().system("pip install colorama")
from colorama import Fore, Back, Style


class ColoredFormatter(logging.Formatter):
    """Colored log formatter."""

    def __init__(self, *args, colors: Optional[Dict[str, str]]=None, **kwargs) -> None:
        """Initialize the formatter with specified format strings."""

        super().__init__(*args, **kwargs)

        self.colors = colors if colors else {}

    def format(self, record) -> str:
        """Format the specified record as text."""

        record.color = self.colors.get(record.levelname, '')
        record.reset = Style.RESET_ALL

        return super().format(record)


formatter = ColoredFormatter(
    '{asctime} |{color} {levelname:8} {reset}| ln-{lineno} |{color} {message}',
    # '{asctime} |{color} {levelname:8} {reset}| {name} | {message}',
    style='{', datefmt='%Y-%m-%d %H:%M:%S',
    colors={
        'DEBUG': Fore.CYAN,
        'INFO': Fore.GREEN,
        'WARNING': Fore.YELLOW,
        'ERROR': Fore.RED,
        'CRITICAL': Fore.RED + Back.WHITE + Style.BRIGHT,
    }
)


In [0]:
!pwd
# !ls /content/drive/'My Drive'/data
import re
import logging
import platform
try: 
  import logzero
except ModuleNotFoundError:
  get_ipython().system("pip install logzero")
import logzero
from logzero import logger
# formatter = logging.Formatter('%(levelname)s:%(lineno)s: %(message)s');
logzero.formatter(formatter)
logzero.loglevel(10)

node = platform.node()
if re.findall(r"[\da-z]{11,}", node):
  loc = "colab"
elif "hamming" in node:
  loc = "win10"

if loc in ['colab']:
  from pathlib import Path
  if not Path("/content/drive/My Drive/data").exists():
    from google.colab import drive
    !mkdir -p /contend/drive
    try:
      drive.mount("/countent/drive")
    except Exception as exc:
      logger.error("drive.mount(\"/countent/drive\")  exc: %s", exc)
logger.info("pwd: %s", os.getcwd())

In [0]:
import os
# %%bash
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES = 256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [0]:
# win10 playground neural-machine-translation load.py

# get_ipython().system('rm %s' % filename)

# http://www.manythings.org/anki/deu-eng.zip
lang ="cmn"
filename = f"{lang}-eng.zip"
os.chdir(r"/content/drive/My Drive/data")
if not Path(filename).exists():
  !wget -c http://www.manythings.org/anki/{filename}
# !curl -O http://www.manythings.org/anki/{filename}
# !wget -c http://www.manythings.org/anki/{filename}

# get_ipython().system('ls')
# get_ipython().system('cp -f deu-eng.zip drive/My\ Drive/data')
# get_ipython().system('ls /content/drive/My\ Drive/data')
# !unzip -tv {filename}

In [0]:
from zipfile import ZipFile
from pathlib import Path

lang = 'deu'
lang = 'cmn'
filename = f'{lang}-eng.zip'


In [0]:
dir_name = '.'
if Path('/content/drive/My Drive/data').exists():
  dir_name = '/content/drive/My Drive/data'

zfilename = f'{dir_name}/{lang}-eng.zip'
# !unzip -t "{zfilename}"

input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0

assert Path(zfilename).exists(), f'{zfilename} does not exits.'
from pprint import pprint
filename = f'{lang}.txt'

with ZipFile(zfilename) as zipfile, zipfile.open(filename) as fha: tot_lines = len([*fha])
# tot_lines = len([*fha])  # 204574
# logger.debug("tot_lines: %s", tot_lines)
# tot_lines = 204574  # lang = 'de'
# lang: cmn, tot_lines: 22075

logger.info("lang: %s, tot_lines: %s, zfilename: %s", lang, tot_lines, zfilename)


In [0]:

break_on = 10  # tot_lines
break_on = NUM_SENTENCES
with ZipFile(zfilename) as zipfile, zipfile.open(filename) as fha:
    # zipfile.filelist:  filename='deu.txt', filename='_about.txt'
    for idx, line in enumerate(fha):
        if idx > break_on - 1: break
        split = line.decode().split(sep="\t")
        if len(split) > 1:
          # en, de, *_ = split 
          entry1, entry2, *_ = split
          # pprint([en, de, ])
        # pprint(line.decode())

        input, output = entry2, entry1
        input_sentences.append(input)
        output_sentences.append(output + ' <eos>')
        output_sentences_inputs.append('<sos> ' + output)

In [0]:
import reprlib
# logger.debug("input_sentences[:3]: %s, input_sentences[:3]: %s, output_sentences_inputs[:3]: %s", input_sentences[:3], output_sentences_inputs[:3], output_sentences[:3])
print('input_sentences:',reprlib.repr(input_sentences))
print('output_sentences:',reprlib.repr(output_sentences))
print('output_sentences_inputs:',reprlib.repr(output_sentences_inputs))

In [0]:
import reprlib
logzero.loglevel(10)
logger.debug(reprlib.repr(input_sentences))
logger.debug(reprlib.repr(output_sentences_inputs))
logger.debug(reprlib.repr(output_sentences))
logger.debug("len(input_sentences): %s", len(input_sentences))
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

print(172, input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

In [0]:
# input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
if lang in ['cmn']:
    input_tokenizer = Tokenizer(
        num_words=MAX_NUM_WORDS,
        char_level=True,
        oov_token='',
    )
else:
    input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

logger.debug("MAX_NUM_WORDS: %s", MAX_NUM_WORDS)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

idx2word_input = {v:k for k, v in word2idx_inputs.items()}
# idx2word_target = {v:k for k, v in word2idx_outputs.items()}

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

# [D 200508 14:12:57 <ipython-input-25-8209ac589f74>:12] MAX_NUM_WORDS: 20000
# Total unique words in the input: 3271
# Length of longest sentence in input: 23

In [0]:
def idx2word(idx, word2idx=word2idx_inputs):
  return [k for k, v in word2idx.items() if v == idx]
in_len_list = [len(_) for _ in input_integer_seq]
argmax_in, max_in = np.argmax(in_len_list), np.max(in_len_list)
logger.debug("argmax_in: %s, max_in: %s, %s", argmax_in, max_in, input_integer_seq[argmax_in])
logger.debug("longest input: %s, ", [idx2word(_, word2idx_inputs) for _ in input_integer_seq[argmax_in]])
''.join([''.join(_) for _ in [idx2word(_, word2idx_inputs) for _ in input_integer_seq[argmax_in]]])
# DEBUG:5: argmax_in: 16521, max_in: 23, [12, 6, 13, 150, 18, 4, 83, 116, 35, 44, 194, 6, 4, 215, 171, 621, 10, 93, 134, 4, 215, 171, 2]
# DEBUG:6: longest input: [['在'], ['你'], ['有'], ['需'], ['要'], ['的'], ['時'], ['候'], ['，'], ['會'], ['幫'], ['你'], ['的'], ['朋'], ['友'], ['才'], ['是'], ['真'], ['正'], ['的'], ['朋'], ['友'], ['。']], 
# '在你有需要的時候，會幫你的朋友才是真正的朋友。'

In [0]:
[''.join(_) for _ in [idx2word(_, word2idx_inputs) for _ in range(15)]]

In [0]:
logger.debug("input_integer_seq: %s", reprlib.repr(input_sentences))
logger.debug("input_integer_seq: %s", reprlib.repr(input_integer_seq))
# [D 200508 14:13:26 <ipython-input-26-bb9286a73bf7>:1] input_integer_seq: ['嗨。', '你好。', '你用跑的。', '等等！', '等一下！', '你好。', ...]
# [D 200508 14:13:26 <ipython-input-26-bb9286a73bf7>:2] input_integer_seq: [[1879, 2], [6, 31, 2], [6, 103, 394, 4, 2], [192, 192, 164], [192, 11, 47, 164], [6, 31, 2], ...]

In [0]:
def idx2word(idx, word2idx=word2idx_inputs):
  return [k for k, v in word2idx.items() if v == idx]
# for sent in [[1879, 2], [6, 31, 2], [6, 103, 394, 4, 2], [192, 192, 164], [192, 11, 47, 164], [6, 31, 2],]:
#   print(sent, [idx2word(elm) for elm in sent])
for idx, sent in  enumerate(input_integer_seq[:20]): 
    print(input_sentences[idx], sent, [idx2word(elm) for elm in sent])

In [0]:
# ##################
# output_sequences
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))
# idx2word_input = {v:k for k, v in word2idx_inputs.items()}
# idx2word_target = {v:k for k, v in word2idx_outputs.items()}
idx2word_outputs = {v:k for k, v in word2idx_outputs.items()}

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)
# en Total unique words in the output: 9102
# en Length of longest sentence in the output: 13

_ = """
Total unique words in the output: 5290
Length of longest sentence in the output: 6

In [430]: [(k, v) for k, v in word2idx_outputs.items() if v==1]
Out[430]: [('<eos>', 1)]
In [431]: [(k, v) for k, v in word2idx_outputs.items() if v==0]
Out[431]: []
[('<sos>', 2)]/[('i', 3)]/[('the', 4)]
[('inner', 9102)]

In [436]: [(k, v) for k, v in word2idx_outputs.items() if v==9103]
Out[436]: []

Total unique words in the output: 9025
Length of longest sentence in the output: 13
# """


In [0]:

encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

In [0]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])
print([idx2word(elm, word2idx_outputs) for elm in [2, 173, 475]])
# [('<sos>', 2)] [('let', 145)] [('me', 24)]  [('in.', 482)]
# -> <sos> let me in.
# decoder_input_sequences.shape: (20000, 13)  # 20000 sents of length 13
# decoder_input_sequences[172]: [  2 173 475   0   0   0   0   0   0   0   0   0   0]
# [['<sos>'], ['keep'], ['them.']]

In [0]:
print(word2idx_outputs['ill'])  # 3446
out_len_list = [len(_) for _ in output_input_integer_seq]
argmax = np.argmax(out_len_list)  # 19929 sos i will give you a call as soon as i get home.
logger.debug("\n\tdecoder_input_sequences: %s, %s", 
  output_input_integer_seq[argmax:argmax + 1], argmax)
logger.debug("\n\t %s", [idx2word(_, word2idx_outputs) for _ in output_input_integer_seq[argmax]])

In [0]:

print(reprlib.repr(word2idx_outputs), len(word2idx_outputs))
def idx2word(idx, word2idx):
  return [k for k, v in word2idx.items() if v == idx]
print([idx2word(elm, word2idx_outputs) for elm in range(1, 20 + 1)], range(1, 20 + 1))

In [0]:
# embedding 
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

# glove.6B.zip http://nlp.stanford.edu/data/glove.6B.zip

if loc in ['colab']:
  glove_file_loc = r"/content/drive/My Drive/data/glove.6B.100d.txt"
  if not Path(glove_file_loc).exists():
    os.chdir(r"/content/drive/My Drive/data")
    ! wget -c http://nlp.stanford.edu/data/glove.6B.zip
    !unzip glove.6B.zip glove.6B.100d.txt
    os.chdir("/content")
else:
  glove_file_loc = r"file_loc"
assert Path(glove_file_loc).exists(), f" file [{glove_file_loc}] does not exists."

# glove_file = open(r'/content/drive/My Drive/data/glove.6B.100d.txt', encoding="utf8")
with open(r'/content/drive/My Drive/data/glove.6B.100d.txt', encoding="utf8") as glove_file:
  for line in glove_file:
      records = line.split()
      word = records[0]
      vector_dimensions = asarray(records[1:], dtype='float32')
      embeddings_dictionary[word] = vector_dimensions
# glove_file.close()

In [0]:
# glove_file = open(r'/content/drive/My Drive/data/glove.6B.100d.txt', encoding="utf8")
# line = next(glove_file)
# print(line)  # the -0.038194 -0.24487 0.72812 -0.39961 0.083172 
# glove_file.close()

for inputs: Recall that we have 3523 unique words in the input. We will create a matrix where the row number will represent the integer value for the word and the columns will correspond to the dimensions of the word. This matrix will contain the word embeddings for the words in our input sentences.

For outputs

In [0]:
# https://stackabuse.com/python-for-nlp-neural-machine-translation-with-seq2seq-in-keras/
num_words = min(MAX_NUM_WORDS, len(word2idx_outputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_outputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector 

In [0]:
index = 0
logger.debug("embedding_matrix.shape: %s", embedding_matrix.shape)  # 9025 + 1, (9026, 100)
logger.debug(" embedding_matrix[0]: %s, len: %s", embedding_matrix[0], len(embedding_matrix[0]))  # all zeros

In [0]:
print(embeddings_dictionary["ill"][:10])
print(embeddings_dictionary["ill"][-10:])
# [ 0.12648 0.1366 0.22192 -0.025204 -0.7197 0.66147 0.48509 0.057223 0.13829 -0.26375 ]
# ...
# [-0.37322 0.50538   0.59171   0.46534  -0.42482   0.83265 0.081548 -0.44147 -0.084311 -1.2304  ]  -

In [0]:
idx_ill = word2idx_outputs['ill']  # 3446
emd_ill = embedding_matrix[idx_ill]
print(emd_ill[:5], '\n', emd_ill[-5:])
# [ 0.12648     0.1366      0.22192    -0.025204   -0.71969998] 
# [ 0.83265001  0.081548   -0.44147    -0.084311   -1.23039997]  # check

In [0]:
logger.debug("EMBEDDING_SIZE: %s", EMBEDDING_SIZE)
assert embedding_matrix.shape[1] == EMBEDDING_SIZE  # (9026, 100)[1] == 100

In [0]:
# embedding layer
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [0]:
# print(embedding_layer)  # keras.layers.embeddings.Embedding object 

## Creating the Model
Therefore, the final shape of the output will be:
```shell
(number of inputs, length of the output sentence, the number of words in the output)
```

In [0]:
import numpy as np
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)

In [0]:
# print(word2idx_inputs["i'm"])  # in the article eng-fra 
# print(word2idx_inputs["ill"])  # 6 539

print(word2idx_outputs["i'm"])
print(word2idx_outputs["ill"])
# 31, 3446
idx2word(31, word2idx_outputs), idx2word(3446, word2idx_outputs), idx2word(31, word2idx_inputs)

In [0]:
logger.debug("decoder_targets_one_hot.shape: %s", decoder_targets_one_hot.shape)  
# eng-cnm swapped: out is en (20000, 13, 9026): 20000 sents, sent lenght 13, vocabs 9026

In [0]:
# from comments john lee
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
print(repr(decoder_output_sequences[:3]))  # 3466,1 (hi., eos), 2276,1, run. eos 

In [0]:

for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1
from pprint import pprint
t0 = decoder_targets_one_hot[:1]
print(t0.shape, t0[0, 0, 3466], t0[0, 1, 1], idx2word(3466, word2idx_outputs))

In [0]:
# encoder
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [0]:
# the decoder LSTM
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [0]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
# The next step is to compile the model:

model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [0]:
# Let's plot our model to see how it looks:

from keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [0]:
# train the model using the fit() method:

r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    # epochs=1,
    epochs=EPOCHS,
    validation_split=0.1,
)

## Modifying the Model for Predictions

**While training**

`// Inputs on the left of Encoder/Decoder, outputs on the right.`

* step 1: -
  I'm ill -> Encoder -> enc(h1,c1)

enc(h1,c1) + <sos> -> Decoder -> je + dec(h1,c1)

* step 2:

enc(h1,c1) + je -> Decoder -> suis + dec(h2,c2)

* step 3:

enc(h2,c2) + suis -> Decoder -> malade. + dec(h3,c3)

* step 3:

enc(h3,c3) + malade. -> Decoder -> <eos> + dec(h4,c4) 


The encoder model remains the same:

In [0]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

**During predictions**



```
// Inputs on the left of Encoder/Decoder, outputs on the right.

Step 1:

I'm ill -> Encoder -> enc(h1,c1)

enc(h1,c1) + <sos> -> Decoder -> y1(je) + dec(h1,c1)

step 2:

enc(h1,c1) + y1 -> Decoder -> y2(suis) + dec(h2,c2)

step 3:

enc(h2,c2) + y2 -> Decoder -> y3(malade.) + dec(h3,c3)

step 3:

enc(h3,c3) + y3 -> Decoder -> y4(<eos>) + dec(h4,c4)
```



In [0]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [0]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [0]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [0]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [0]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [0]:
from keras.utils import plot_model
plot_model(decoder_model, to_file='model_plot_dec.png', show_shapes=True, show_layer_names=True)

## Making Predictions

In [0]:
idx2word_input = {v: k for k, v in word2idx_inputs.items()}
idx2word_target = {v: k for k, v in word2idx_outputs.items()}

In [0]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

## Testing the Model

In [0]:
import numpy as np
logzero.loglevel(10)

i = np.random.choice(len(input_sentences))
logger.info("input_sentences %s", i)
input_seq = encoder_input_sequences[i:i+1]
logger.debug("input_seq %s", input_seq)
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

## Concluding

## Other Misc Notes

In [0]:
# ############## misc notes
import logging
logger = logging.getLogger(__name__)
# coloredlogs
try:
  import coloredlogs
except ModuleNotFoundError:
  get_ipython().system("pip install coloredlogs")
  import coloredlogs

coloredlogs.install(level='DEBUG')
logger.debug(reprlib.repr(input_sentences))
logger.debug(reprlib.repr(output_sentences_inputs))
logger.debug(reprlib.repr(output_sentences))

In [0]:
# https://gist.github.com/joshbode/58fac7ababc700f51e2a9ecdebe563ad
import sys
import logging
from typing import Optional, Dict

try:
  import colorama
except ModuleNotFoundError:
  get_ipython().system("pip install colorama")
from colorama import Fore, Back, Style


class ColoredFormatter(logging.Formatter):
    """Colored log formatter."""

    def __init__(self, *args, colors: Optional[Dict[str, str]]=None, **kwargs) -> None:
        """Initialize the formatter with specified format strings."""

        super().__init__(*args, **kwargs)

        self.colors = colors if colors else {}

    def format(self, record) -> str:
        """Format the specified record as text."""

        record.color = self.colors.get(record.levelname, '')
        record.reset = Style.RESET_ALL

        return super().format(record)


formatter = ColoredFormatter(
    '{asctime} |{color} {levelname:8} {reset}| ln-{lineno} |{color} {message}',
    # '{asctime} |{color} {levelname:8} {reset}| {name} | {message}',
    style='{', datefmt='%Y-%m-%d %H:%M:%S',
    colors={
        'DEBUG': Fore.CYAN,
        'INFO': Fore.GREEN,
        'WARNING': Fore.YELLOW,
        'ERROR': Fore.RED,
        'CRITICAL': Fore.RED + Back.WHITE + Style.BRIGHT,
    }
)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)

logger = logging.getLogger(__name__)
logger = logging.getLogger()
logger.handlers[:] = []
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)
#logger.setLevel(logging.INFO)

logger.debug("debug")
logger.info("info: %s", 'info')
logger.warning("warning")
logger.error("erorr")
from logzero import logger
logzero.loglevel(20)
logzero.formatter(formatter)
logger.debug('ddd')
logger.info('iii')